# 색 추출기
- 실행순서
1. 드라이브 마운트 하기(코랩을 이용하지 않는다면 pass)
2. 준비코드에 보면 디렉토리 변경 코드가 있다(os.chdir). 이것을 Clothes-Color-Extraction 디렉토리로 변경한다.
3. 맨 아래 셀에 실행 코드가 있다. image에 색을 추출하고자 하는 이미지 이름*을 넣고 실행시키면 된다.
* 이미지의 위치가 2번에서 수정된 디렉토리가 아니라면 디렉토리까지 합쳐서 이름을 넣는다. (ex. '/content/drive/My Drive ... /sample.png')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# 준비 코드
import cv2
import numpy as np
import os
import sys
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten,Dropout
from keras.models import model_from_json
import math
import urllib

###############
os.chdir('/content/drive/My Drive/Colab Notebooks/Clothes-Color-Extraction')
###############

image_width = 330
image_height = 450

def path_file(file):
    return str(file)

def predictor(img_file):
    img = cv2.imread(img_file)
    resize = cv2.resize(img,(64,64))
    #print(img)
    img_fin = np.reshape(resize,[1,64,64,3])
    json_file = open('model/binaryfas10.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights("model/binaryfashion.h5")
    prediction = loaded_model.predict_classes(img_fin)
    prediction = np.squeeze(prediction,axis=1)
    predict = np.squeeze(prediction,axis=0)
    return int(predict)

def seg(img_file):
    predict = predictor(img_file)
    file = path_file("annotation.csv")
    reader = pd.read_csv(file)
    img = cv2.imread(img_file)
    img = cv2.resize(img,(image_width,image_height))
    mask = np.zeros(img.shape[:2],np.uint8)   
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
    rect = (reader.x1[predict],reader.y1[predict],reader.x2[predict],reader.y2[predict])
    cv2.grabCut(img,mask,rect,bgdModel,fgdModel,reader.i[predict],cv2.GC_INIT_WITH_RECT)
    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    img_cut = img*mask2[:,:,np.newaxis]
    #cv2.imwrite('seg_' + img_file,img_cut) 
    return img_cut

# 색 리스트를 불러옵니다
color_list = pd.read_excel('Colors.xlsx')
red_list = color_list[['RED', 'R', 'G', 'B']]
color_df = []
for i in range(len(red_list)):
    temp = [red_list['RED'][i], color_list['R'][i], color_list['G'][i], color_list['B'][i]]
    color_df.append(temp)
red_list = color_df

orange_list = color_list[['ORANGE', 'R.1', 'G.1', 'B.1']]
color_df = []
for i in range(len(orange_list)):
    temp = [orange_list['ORANGE'][i], color_list['R.1'][i], color_list['G.1'][i], color_list['B.1'][i]]
    color_df.append(temp)
orange_list = color_df

yellow_list = color_list[['YELLOW', 'R.2', 'G.2', 'B.2']]
color_df = []
for i in range(len(yellow_list)):
    temp = [yellow_list['YELLOW'][i], color_list['R.2'][i], color_list['G.2'][i], color_list['B.2'][i]]
    color_df.append(temp)
yellow_list = color_df

green_list = color_list[['GREEN', 'R.3', 'G.3', 'B.3']]
color_df = []
for i in range(len(green_list)):
    temp = [green_list['GREEN'][i], color_list['R.3'][i], color_list['G.3'][i], color_list['B.3'][i]]
    color_df.append(temp)
green_list = color_df

blue_list = color_list[['BLUE', 'R.4', 'G.4', 'B.4']]
color_df = []
for i in range(len(blue_list)):
    temp = [blue_list['BLUE'][i], color_list['R.4'][i], color_list['G.4'][i], color_list['B.4'][i]]
    color_df.append(temp)
blue_list = color_df

purple_list = color_list[['PURPLE', 'R.5', 'G.5', 'B.5']]
color_df = []
for i in range(len(purple_list)):
    temp = [purple_list['PURPLE'][i], color_list['R.5'][i], color_list['G.5'][i], color_list['B.5'][i]]
    color_df.append(temp)
purple_list = color_df

magenta_list = color_list[['MAGENTA', 'R.6', 'G.6', 'B.6']]
color_df = []
for i in range(len(green_list)):
    temp = [magenta_list['MAGENTA'][i], color_list['R.6'][i], color_list['G.6'][i], color_list['B.6'][i]]
    color_df.append(temp)
magenta_list = color_df

# 배경을 제거합니다
def del_bg(image):
    background = []
    count = 0
    for i in image:
        if i[0] == 0 and i[1] == 0 and i[2] == 0:
            background.append(count)
        count += 1
    bg_deleted = np.delete(image, background, axis = 0)
    return bg_deleted

# 무채색과 유채색으로 나눕니다
def which_chromatic(image):
    achromatic = []
    chromatic = []
    for cl in image: 
        max_value = max(cl[0], cl[1], cl[2])
        min_value = min(cl[0], cl[1], cl[2])
        if (max_value - min_value) <= 15:
            achromatic.append(cl.tolist())
        else:
            chromatic.append(cl.tolist())
    return achromatic, chromatic

# 무채색을 검정색, 회색, 흰색으로 나눕니다
def categorize_achromatic(x_achromatic):
    black = []
    grey = []
    white = []
    for i in x_achromatic:
        if (i[0]+i[1]+i[2])/3 >= 0 and (i[0]+i[1]+i[2])/3 <71: # 검정색
            black.append(i)
        if (i[0]+i[1]+i[2])/3 >= 71 and (i[0]+i[1]+i[2])/3 <120: # 어두운 회색
            grey.append(i)
        if (i[0]+i[1]+i[2])/3 >= 120 and (i[0]+i[1]+i[2])/3 <172: # 회색
            grey.append(i)
        if (i[0]+i[1]+i[2])/3 >= 172 and (i[0]+i[1]+i[2])/3 <200: # 밝은 회색
            grey.append(i)
        if (i[0]+i[1]+i[2])/3 >= 200 and (i[0]+i[1]+i[2])/3 <255: # 흰색
            white.append(i)
    return black, grey, white

# RGB를 HSV로 변환합니다
def rgb_to_hsv(r, g, b):
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    df = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        h = (60 * ((g-b)/df) + 360) % 360
    elif mx == g:
        h = (60 * ((b-r)/df) + 120) % 360
    elif mx == b:
        h = (60 * ((r-g)/df) + 240) % 360
    if mx == 0:
        s = 0
    else:
        s = (df/mx)*100
    v = mx*100
    return round(h, 2), round(s, 2), round(v, 2)

# 유채색의 RGB값들을 HSV로 변환합니다
def chromatic_to_hsv(x_chromatic):
    chromatic_hsv = []
    for i in x_chromatic:
        r = i[0]
        g = i[1]
        b = i[2]
        chromatic_hsv.append(rgb_to_hsv(r, g, b))
    temp = []
    c = 0
    for i in chromatic_hsv:
        temp.append(list(i))
    chromatic_hsv = temp
    return chromatic_hsv

# 유채색을 색상별로 분류합니다
def categorize_chromatic(x_chromatic_hsv):
    red = []
    orange = []
    yellow = []
    green = []
    blue = []
    purple = []
    magenta = []
    red2 = []
    orange2 = []
    yellow2 = []
    green2 = []
    blue2 = []
    purple2 = []
    magenta2 = []
    for i in x_chromatic_hsv:
        if i[0] >= 345 and i[0] <360: # 빨강색
            red.append(i)
        if i[0] >= 0 and i[0] <15: # 빨강색
            red2.append(i)
        if i[0] >= 15 and i[0] <30: # 주황색
            orange.append(i)
        if i[0] >= 30 and i[0] <45: # 주황색
            orange2.append(i)
        if i[0] >= 45 and i[0] <60: # 노랑색
            yellow.append(i)
        if i[0] >= 60 and i[0] <75: # 노랑색
            yellow2.append(i)
        if i[0] >= 75 and i[0] <112.5: # 초록색
            green.append(i)
        if i[0] >= 112.5 and i[0] <150: # 초록색
            green2.append(i)            
        if i[0] >= 150 and i[0] <202.5: # 파랑색
            blue.append(i)
        if i[0] >= 202.5 and i[0] <255: # 파랑색
            blue2.append(i)            
        if i[0] >= 255 and i[0] <277.5: # 보라색
            purple.append(i)
        if i[0] >= 277.5 and i[0] < 300: # 보라색
            purple2.append(i)        
        if i[0] >= 300 and i[0] <322.5: # 자주색
            magenta.append(i)
        if i[0] >= 322.5 and i[0] <345: # 자주색
            magenta2.append(i)            
    return red, orange, yellow, green, blue, purple, magenta, red2, orange2, yellow2, green2, blue2, purple2, magenta2

# HSV값을 RGB로 변환합니다
def hsv_to_rgb(h, s, v):
    i = math.floor(h*6)
    f = h*6 - i
    p = v * (1-s)
    q = v * (1-f*s)
    t = v * (1-(1-f)*s)

    r, g, b = [
        (v, t, p),
        (q, v, p),
        (p, v, t),
        (p, q, v),
        (t, p, v),
        (v, p, q),
    ][int(i%6)]
    return r, g, b

# 유채색의 HSV값을 다시 RGB로 변환합니다
def hsv_to_rgb_execute(x_red, x_orange, x_yellow, x_green, x_blue, x_purple, x_magenta, x_red2, x_orange2, x_yellow2, x_green2, x_blue2, x_purple2, x_magenta2):
    red_temp = []
    orange_temp = []
    yellow_temp = []
    green_temp = []
    blue_temp = []
    purple_temp = []
    magenta_temp = []
    red2_temp = []
    orange2_temp = []
    yellow2_temp = []
    green2_temp = []
    blue2_temp = []
    purple2_temp = []
    magenta2_temp = []
    for i in x_red:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        red_temp.append([r*255, g*255, b*255])
    for i in x_orange:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        orange_temp.append([r*255, g*255, b*255])
    for i in x_yellow:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        yellow_temp.append([r*255, g*255, b*255])
    for i in x_green:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        green_temp.append([r*255, g*255, b*255])
    for i in x_blue:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        blue_temp.append([r*255, g*255, b*255])
    for i in x_purple:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        purple_temp.append([r*255, g*255, b*255])
    for i in x_magenta:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        magenta_temp.append([r*255, g*255, b*255])
    for i in x_red2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        red2_temp.append([r*255, g*255, b*255])
    for i in x_orange2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        orange2_temp.append([r*255, g*255, b*255])
    for i in x_yellow2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        yellow2_temp.append([r*255, g*255, b*255])
    for i in x_green2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        green2_temp.append([r*255, g*255, b*255])
    for i in x_blue2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        blue2_temp.append([r*255, g*255, b*255])
    for i in x_purple2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        purple2_temp.append([r*255, g*255, b*255])
    for i in x_magenta2:
        r,g,b = hsv_to_rgb(i[0]/360, i[1]/100, i[2]/100)
        magenta2_temp.append([r*255, g*255, b*255])        
    return red_temp, orange_temp, yellow_temp, green_temp, blue_temp, purple_temp, magenta_temp, red2_temp, orange2_temp, yellow2_temp, green2_temp, blue2_temp, purple2_temp, magenta2_temp

# 가중평균 RGB값을 구합니다
def rgb_average(x_color):
    r_total = float()
    g_total = float()
    b_total = float()
    for i in x_color:
        r = i[0]
        g = i[1]
        b = i[2]
        r_total += r
        g_total += g
        b_total += b
    try:
        color_final = [r_total/len(x_color), g_total/len(x_color), b_total/len(x_color)]
    except:
        color_final = [0, 0, 0]
    return color_final

# 각 색상별로 가중평균 RGB값을 구합니다
def weighted_chromatic(x_red, x_orange, x_yellow, x_green, x_blue, x_purple, x_magenta, x_grey, x_red2, x_orange2, x_yellow2, x_green2, x_blue2, x_purple2, x_magenta2):
    red_final = rgb_average(x_red)
    orange_final = rgb_average(x_orange)
    yellow_final = rgb_average(x_yellow)
    green_final = rgb_average(x_green)
    blue_final = rgb_average(x_blue)
    purple_final = rgb_average(x_purple)
    magenta_final = rgb_average(x_magenta)
    grey_final = rgb_average(x_grey)
    red_final2 = rgb_average(x_red2)
    orange_final2 = rgb_average(x_orange2)
    yellow_final2 = rgb_average(x_yellow2)
    green_final2 = rgb_average(x_green2)
    blue_final2 = rgb_average(x_blue2)
    purple_final2 = rgb_average(x_purple2)
    magenta_final2 = rgb_average(x_magenta2)
    return red_final, orange_final, yellow_final, green_final, blue_final, purple_final, magenta_final, grey_final, red_final2, orange_final2, yellow_final2, green_final2, blue_final2, purple_final2, magenta_final2

# 유채색 각 색상별 색의 이름과 그 비중을 구합니다
def final_chromatic(x_color, x_color_final, x_color_list, x_img):
    c = 0
    min_value_name = str() 
    for i in x_color_list:
        temp = (x_color_final[0] - i[1])**2 + (x_color_final[1] - i[2])**2 + (x_color_final[2] - i[3])**2
        if c == 0:
            min_value = temp
            min_value_name = i[0]
        else:
            if min_value > temp:
                min_value = temp
                min_value_name = i[0]
        c += 1
    return [min_value_name, round(len(x_color)/len(x_img),2)]
        
def Run_color_final(image):
    img = seg(image)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[0] * img.shape[1], 3))
    img = del_bg(img)
    achromatic, chromatic = which_chromatic(img)
    black, grey, white = categorize_achromatic(achromatic)
    chromatic_hsv = chromatic_to_hsv(chromatic)
    red, orange, yellow, green, blue, purple, magenta, red2, orange2, yellow2, green2, blue2, purple2, magenta2 = categorize_chromatic(chromatic_hsv)
    red_rgb, orange_rgb, yellow_rgb,green_rgb, blue_rgb, purple_rgb, magenta_rgb, red_rgb2, orange_rgb2, yellow_rgb2,green_rgb2, blue_rgb2, purple_rgb2, magenta_rgb2  = hsv_to_rgb_execute(red, orange, yellow, green, blue, purple, magenta, red2, orange2, yellow2, green2, blue2, purple2, magenta2)
    red_final, orange_final, yellow_final, green_final, blue_final, purple_final, magenta_final, grey_final, red_final2, orange_final2, yellow_final2, green_final2, blue_final2, purple_final2, magenta_final2 = weighted_chromatic(red_rgb, orange_rgb, yellow_rgb,green_rgb, blue_rgb, purple_rgb, magenta_rgb, grey, red_rgb2, orange_rgb2, yellow_rgb2,green_rgb2, blue_rgb2, purple_rgb2, magenta_rgb2)
    final_colors = []

    # 각 색(색상)별 비중이 전체의 3%이상이면 출력이 되도록 설정해놨습니다.
    if len(black)/len(img)*100 >= 3:
        final_colors.append(['검정색', round(len(black)/len(img),2)])
    if len(grey)/len(img)*100 >= 3:
        if (grey_final[0]+grey_final[1]+grey_final[2])/3 >= 71 and (grey_final[0]+grey_final[1]+grey_final[2])/3 <120: # 어두운 회색
            final_colors.append(['어두운 회색', round(len(grey)/len(img),2)])
        if (grey_final[0]+grey_final[1]+grey_final[2])/3 >= 120 and (grey_final[0]+grey_final[1]+grey_final[2])/3 <172: # 회색
            final_colors.append(['회색', round(len(grey)/len(img),2)])
        if (grey_final[0]+grey_final[1]+grey_final[2])/3 >= 172 and (grey_final[0]+grey_final[1]+grey_final[2])/3 <240: # 밝은 회색
            final_colors.append(['밝은 회색', round(len(grey)/len(img),2)])
    if len(white)/len(img)*100 >= 3:
        final_colors.append(['흰색', round(len(white)/len(img),2)])
    if len(red)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(red, red_final, red_list, img))
    if len(orange)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(orange, orange_final, orange_list, img))
    if len(yellow)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(yellow, yellow_final, yellow_list, img))
    if len(green)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(green, green_final, green_list, img))
    if len(blue)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(blue, blue_final, blue_list, img))
    if len(purple)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(purple, purple_final, purple_list, img))
    if len(magenta)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(magenta, magenta_final, magenta_list, img))
    if len(red2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(red2, red_final2, red_list, img))
    if len(orange2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(orange2, orange_final2, orange_list, img))
    if len(yellow2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(yellow2, yellow_final2, yellow_list, img))
    if len(green2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(green2, green_final2, green_list, img))
    if len(blue2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(blue2, blue_final2, blue_list, img))
    if len(purple2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(purple2, purple_final2, purple_list, img))
    if len(magenta2)/len(img)*100 >= 3:
       final_colors.append(final_chromatic(magenta2, magenta_final2, magenta_list, img))
    final_colors.sort(key = lambda x:x[1], reverse = True)
    clothes_color = final_colors
    print(clothes_color)

In [4]:
# 실행코드

## 색을 추출하고자 하는 이미지
image = 'sample.png'

## 추출 실행
Run_color_final(image)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[['남색', 1.0]]
